In [ ]:
!pip install Keras-Preprocessing
!pip install pyvi 
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import keras.models
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras_preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re 
from unidecode import unidecode

In [ ]:
path = '/content/Data.csv'
data = pd.read_csv(path)
data = data[['Review','Label']]
print(data[['Review','Label']])
print()
print("data['Label'] == '1': ", (data['Label'] == -1).sum())
print("data['Label'] == '-1': ", (data['Label'] == -1).sum())

                                                 Review  Label
0     Lần đầu_tiên ăn chắc cũng là lần cuối ăn_ở đây...     -1
1     Khi mình vào thì bàn chưa dọn , dơ kinh . Cái ...     -1
2     Haiz ! Không biết đúng hương_vị của bánh thế_n...     -1
3     Mình ghé quán này vì thấy có cơm theo kiểu Việ...     -1
4     Quán xịn ở Quan_Hoa - Cầu_Giấy , biển_hiệu màu...     -1
...                                                 ...    ...
9995  Quán không_gian nhỏ nhưng khá lịch_sự , sạch_s...      1
9996  đi ăn lần 2 : ) ) ) lần này rút kinh_nghiệm ki...      1
9997  Mình ăn 2 món , mà quên tên mất rồi , toàn từ ...      1
9998  Trong ảnh là\n• 1 phần mì udon xào\n• 1 phần c...      1
9999  địa_điểm : dễ tìm , mậu thân quẹo vào đường bờ...      1

[10000 rows x 2 columns]

data['Label'] == '1':  5000
data['Label'] == '-1':  5000


In [ ]:

print("data['Label'] == '1': ", (data['Label'] == '-1').sum())
print("data['Label'] == '-1': ", (data['Label'] == '-1').sum(), '\n')

data['Label'] = data['Label'].astype(str)

data['Review'] = data['Review'].apply(lambda x: x.lower())
print(f"{'lower: ':<13}", data.iloc[0]['Review'])

data['Review'] = data['Review'].apply((lambda x: re.sub(r'[^a-zA-Z0-9\sÀ-ỹ]',' ',x)))
print(f"{'resub: ':<13}", data.iloc[0]['Review'])

data['Review'] = data['Review'].apply(lambda x: unidecode(x))
print(f"{'unidecode: ':<13}" , data.iloc[0]['Review'])

for idx,row in data.iterrows():
  row[0] = row[0].replace('rt',' ')
print(f"{'finall: ':<13}" , data.iloc[0]['Review'])
max_fatures =2000
X = data['Review'].values #//posts
tokenizer: Tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Review'].values)
X = tokenizer.texts_to_sequences(data['Review'].values)
X = pad_sequences(X)


data['Label'] == '1':  0
data['Label'] == '-1':  0 

lower:        lần đầu_tiên ăn chắc cũng là lần cuối ăn_ở đây quá , món ăn thì không có gì đặc_sắc , giá_cả thì mắc , đồ_ăn thì không được nhìu . nói_chung là mình thấy bình_thường so với một nhà_hàng tầm_cỡ như_vậy . thất_vọng ! ! ! !

resub:        lần đầu tiên ăn chắc cũng là lần cuối ăn ở đây quá   món ăn thì không có gì đặc sắc   giá cả thì mắc   đồ ăn thì không được nhìu   nói chung là mình thấy bình thường so với một nhà hàng tầm cỡ như vậy   thất vọng        

unidecode:    lan dau tien an chac cung la lan cuoi an o day qua   mon an thi khong co gi dac sac   gia ca thi mac   do an thi khong duoc nhiu   noi chung la minh thay binh thuong so voi mot nha hang tam co nhu vay   that vong        

finall:       lan dau tien an chac cung la lan cuoi an o day qua   mon an thi khong co gi dac sac   gia ca thi mac   do an thi khong duoc nhiu   noi chung la minh thay binh thuong so voi mot nha hang tam co nhu vay   that vong        



In [ ]:
Y = pd.get_dummies(data['Label']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3,
                                                    random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)


(7000, 1582) (7000, 2)
(3000, 1582) (3000, 2)


In [ ]:
embed_dim = 128
lstm_out = 196
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1582, 128)         256000    
                                                                 
 lstm_1 (LSTM)               (None, 196)               254800    
                                                                 
 dense_1 (Dense)             (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:

batch_size = 32
model.fit(X_train,Y_train,batch_size=batch_size,epochs=7,verbose =1 )

Epoch 1/7
219/219 [==============================] - 1577s 7s/step - loss: 0.4919 - accuracy: 0.7670
Epoch 2/7
219/219 [==============================] - 1562s 7s/step - loss: 0.3629 - accuracy: 0.8513
Epoch 3/7
219/219 [==============================] - 1559s 7s/step - loss: 0.3017 - accuracy: 0.8764
Epoch 4/7
219/219 [==============================] - 1561s 7s/step - loss: 0.2827 - accuracy: 0.8859
Epoch 5/7
219/219 [==============================] - 1559s 7s/step - loss: 0.2384 - accuracy: 0.9036
Epoch 6/7
219/219 [==============================] - 1543s 7s/step - loss: 0.2204 - accuracy: 0.9103
Epoch 7/7
219/219 [==============================] - 1533s 7s/step - loss: 0.2003 - accuracy: 0.9220


In [ ]:
import pickle

#tạo model model_LSTM_v310523 đặc trưng data train tiếng Việt ko dấu ('dau tien', 'an o'..)
#cập nhật: ...
pkl_filename = "/content/drive/Othercomputers/My Laptop /Do_An/Do_An_2/demo/model_LSTM_v310523.pkl"
with open(pkl_filename, 'wb') as file:
  pickle.dump(model, file)

with open(pkl_filename, 'rb') as file:
  load_model_LSTM = pickle.load(file)